In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from Scratch import metadata_loader
import numpy as np
from tqdm import tqdm

DEVICE = "ibm_sherbrooke"
LOGICAL = str(0)
XBASIS = False
ROUNDS = 100

state = "X" if XBASIS else "Z"
state += LOGICAL
print(f"state = {state}")


# Load the metadata
md = metadata_loader(True, True)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["code"] == "RepetitionCodeCircuit"]
md = md[md["descr"] == 'subset RepCodes']
md = md.dropna(subset=["rounds"])
md = md[md["meas_level"] == 1]
md['rounds'] = md['rounds'].astype(int)
md['distance'] = md['distance'].astype(int)

md = md[md["backend_name"] == DEVICE]
md = md[md["logical"] == LOGICAL]
md = md[md["xbasis"] == XBASIS]
md = md[md["rounds"] == ROUNDS]

md = md[md['notebook_name']=='Sherbrooke_RepCodes_jobs']

# md = md[0:1]
print("shape:", md.shape)
md[:2]


state = Z0
shape: (40, 18)


,creation_date,notebook_name,backend_name,job_id,tags,meas_level,shots,num_qubits,job_status,execution_date,code,distance,rounds,logical,descr,resets,xbasis,path_info
5259,2024-03-23 17:44:10.530000+01:00,Sherbrooke_RepCodes_jobs,ibm_sherbrooke,cqzgdppczq6g0081hjfg,"[Subset 52, 100 rounds, 0 log, xbasis=False]",1.0,768.0,NaN,JobStatus.DONE,2024-03-23 18:05:17.770180+01:00,RepetitionCodeCircuit,52,100,0,subset RepCodes,NaN,False,"{'mean_gate_error': 0.008345314216382, 'min_ga..."
5258,2024-03-23 17:43:48.780000+01:00,Sherbrooke_RepCodes_jobs,ibm_sherbrooke,cqzgdh6ktf3g00883tn0,"[Subset 52, 100 rounds, 0 log, xbasis=False]",1.0,768.0,NaN,JobStatus.DONE,2024-03-23 18:04:47.231991+01:00,RepetitionCodeCircuit,52,100,0,subset RepCodes,NaN,False,"{'mean_gate_error': 0.008345314216382, 'min_ga..."


In [3]:
from result_saver import SaverProvider

provider = SaverProvider()
md_loc = 0

job_id = md.iloc[md_loc]["job_id"]
job = provider.retrieve_job(job_id) 

execution_date = job.result().date
memory = job.result().get_memory()

print(f"Execution date: {execution_date}")

Execution date: 2024-03-23 18:05:17.770180+01:00


In [4]:
from soft_info import inv_qubit_mapping, get_repcode_IQ_map

_RESETS = False
SOFT_MUL = 0.5
T = md.iloc[md_loc]["rounds"]
d = md.iloc[md_loc]["distance"]

shots = md.iloc[md_loc]["shots"]
# Get the layout of the last job (same as previous)
layout_des = job.deserialize_layout(job.initial_layouts()[0]) # only 1 layout
link_qubits = list(layout_des['link_qubit'].values())
code_qubits = list(layout_des['code_qubit'].values())

# Get the pSoft and countMat matrices
big_layout = link_qubits + code_qubits
inverted_q_map = inv_qubit_mapping(get_repcode_IQ_map(big_layout, synd_rounds=T))

In [5]:
from Scratch import load_calibration_memory

all_memories_PS, _, msmt_err_dict = load_calibration_memory(provider, tobecalib_job=job_id, post_process=True, double_msmt=True)

Specified job execution date: 2024-03-23 18:05:17.770180+01:00
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 17:06:20.263377+00:00.


# RepCode data pipeline

In [15]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

# Generate synthetic data
def generate_data(num_samples):
    # Generating data from three different normal distributions
    data1 = tf.random.normal((num_samples // 3, 2), mean=[2.0, 3.0], stddev=1.0)
    data2 = tf.random.normal((num_samples // 3, 2), mean=[-2.0, -3.0], stddev=1.0)
    data3 = tf.random.normal((num_samples // 3, 2), mean=[4.0, -2.0], stddev=1.0)
    return tf.concat([data1, data2, data3], axis=0)

# Define the GMM model
def build_gmm():
    n_components = 3
    n_dims = 2  # Assuming 2-dimensional data

    # Component means
    component_means = tf.Variable(tf.random.normal([n_components, n_dims]))

    # Component scale diagonals (standard deviations)
    scale_diags = tf.Variable(tf.random.uniform([n_components, n_dims], minval=0.5, maxval=1.5))

    # Mixture weights
    mixture_logits = tf.Variable(tf.zeros([n_components]))

    # Define the Gaussian components
    components = tfd.MultivariateNormalDiag(loc=component_means, scale_diag=scale_diags)

    # Create the mixture model
    mixture = tfd.MixtureSameFamily(
        mixture_distribution=tfd.Categorical(logits=mixture_logits),
        components_distribution=tfd.Independent(components)
    )

    return mixture

@tf.function
def train_step(data, model, optimizer):
    with tf.GradientTape() as tape:
        # Negative log-likelihood to minimize
        nll = -tf.reduce_mean(model.log_prob(data))
    gradients = tape.gradient(nll, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return nll

data = generate_data(300)  # Generate synthetic data
gmm_model = build_gmm()  # Build the GMM
optimizer = tf.optimizers.Adam(learning_rate=0.01)

# Training loop
for step in range(1000):
    nll = train_step(data, gmm_model, optimizer)
    if step % 100 == 0:
        print(f"Step {step}: NLL = {nll.numpy()}")


ValueError: in user code:

    File "/var/folders/_3/n7zzdff5239886c9gmhrk6yr0000gn/T/ipykernel_12018/2336361769.py", line 44, in train_step  *
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    File "/Users/mha/.local/share/virtualenvs/Soft-Info-fMUpUe5a/lib/python3.11/site-packages/keras/src/optimizers/base_optimizer.py", line 282, in apply_gradients  **
        self.apply(grads, trainable_variables)
    File "/Users/mha/.local/share/virtualenvs/Soft-Info-fMUpUe5a/lib/python3.11/site-packages/keras/src/optimizers/base_optimizer.py", line 351, in apply
        self._backend_apply_gradients(grads, trainable_variables)
    File "/Users/mha/.local/share/virtualenvs/Soft-Info-fMUpUe5a/lib/python3.11/site-packages/keras/src/optimizers/base_optimizer.py", line 405, in _backend_apply_gradients
        self._backend_update_step(
    File "/Users/mha/.local/share/virtualenvs/Soft-Info-fMUpUe5a/lib/python3.11/site-packages/keras/src/backend/tensorflow/optimizer.py"

In [12]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

def build_gmm(data):
    n_dims = data.shape[-1]  # Extract the number of dimensions in data

    # Component means
    component_means = tf.Variable(tf.random.normal([3, n_dims]))

    # Unified scale diagonals for each component
    scale_diags = tfp.util.TransformedVariable(
        initial_value=tf.ones([3, n_dims]),
        bijector=tfp.bijectors.Softplus()
    )

    # Mixture weights
    mixture_logits = tf.Variable(tf.zeros([3]))

    # Define the Gaussian mixture components
    components = tfd.MultivariateNormalDiag(loc=component_means, scale_diag=scale_diags)

    # Create the mixture model
    mixture = tfd.MixtureSameFamily(
        mixture_distribution=tfd.Categorical(logits=mixture_logits),
        components_distribution=components
    )

    return mixture

@tf.function
def train_step(data, model, optimizer):
    with tf.GradientTape() as tape:
        # Calculate the negative log-likelihood
        nll = -tf.reduce_mean(model.log_prob(data))
    gradients = tape.gradient(nll, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return nll

# Generate synthetic data
data = tf.random.normal([100, 2])  # 100 samples, each with 2 dimensions

# Build and train the Gaussian Mixture Model
gmm_model = build_gmm(data)
optimizer = tf.optimizers.Adam(learning_rate=0.01)

# Training loop
for step in range(1000):
    nll = train_step(data, gmm_model, optimizer)
    if step % 100 == 0:
        print(f"Step {step}: NLL = {nll.numpy()}")


ValueError: in user code:

    File "/var/folders/_3/n7zzdff5239886c9gmhrk6yr0000gn/T/ipykernel_12018/2502449684.py", line 37, in train_step  *
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    File "/Users/mha/.local/share/virtualenvs/Soft-Info-fMUpUe5a/lib/python3.11/site-packages/keras/src/optimizers/base_optimizer.py", line 282, in apply_gradients  **
        self.apply(grads, trainable_variables)
    File "/Users/mha/.local/share/virtualenvs/Soft-Info-fMUpUe5a/lib/python3.11/site-packages/keras/src/optimizers/base_optimizer.py", line 351, in apply
        self._backend_apply_gradients(grads, trainable_variables)
    File "/Users/mha/.local/share/virtualenvs/Soft-Info-fMUpUe5a/lib/python3.11/site-packages/keras/src/optimizers/base_optimizer.py", line 405, in _backend_apply_gradients
        self._backend_update_step(
    File "/Users/mha/.local/share/virtualenvs/Soft-Info-fMUpUe5a/lib/python3.11/site-packages/keras/src/backend/tensorflow/optimizer.py"